In [13]:
%matplotlib inline

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import  transforms,datasets
from torch.utils import data

In [8]:
categories = !ls Data/speech_commands/train | sort -u 

In [9]:
categories=categories[1:]

In [11]:
category_dict={cat:i for i,cat in enumerate(categories)}

In [12]:
print(category_dict)

{'bed': 0, 'bird': 1, 'cat': 2, 'dog': 3, 'down': 4, 'eight': 5, 'five': 6, 'four': 7, 'go': 8, 'happy': 9, 'house': 10, 'left': 11, 'marvin': 12, 'nine': 13, 'no': 14, 'off': 15, 'on': 16, 'one': 17, 'right': 18, 'seven': 19, 'sheila': 20, 'six': 21, 'stop': 22, 'three': 23, 'tree': 24, 'two': 25, 'up': 26, 'wow': 27, 'yes': 28, 'zero': 29}


In [36]:
partition={}
df=pd.read_csv('Data/speech_commands/csvs/train.csv')
partition['train']=df['file_name'].get_values()
partition['train_labels']=df['label'].get_values()

df=pd.read_csv('Data/speech_commands/csvs/valid.csv')
partition['validation']=df['file_name'].get_values()
partition['validation_labels']=df['label'].get_values()

In [37]:
class AudioDataset(data.Dataset):
    
    def __init__(self,filenames,labels):
        self.filenames=filenames
        self.labels=labels
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self,index):
        x = self.filenames[index]
        y = self.labels[index]
        return x,y

In [38]:
train_dataset=AudioDataset(partition['train'],partition['train_labels'])

In [39]:
train_loader = data.DataLoader(train_dataset,batch_size=32,shuffle=True)

In [42]:
aa=iter(train_loader)
x,y=next(aa)

In [46]:
y.shape

torch.Size([32])